In [9]:
Grouped_data=('Bombero', ([{'evento_id': '51c57837', 'timestamp_evento': '2025-05-02 17:51:04', 'servicio': 'Bombero', 'tipo': 'Individual', 'discapacidad': 'Grado 2: Discapacidad leve', 'nivel_emergencia': 'Nivel 2: Emergencia moderada', 'lat': 39.465235, 'lon': -0.380162, 'edad': 56}, {'evento_id': 'af8c53ce', 'timestamp_evento': '2025-05-02 17:51:40', 'servicio': 'Bombero', 'tipo': 'Individual', 'discapacidad': 'Grado 1: Discapacidad nula', 'nivel_emergencia': 'Nivel 3: Emergencia grave', 'lat': 39.475542, 'lon': -0.394527, 'edad': 77}, {'evento_id': '90a17027', 'timestamp_evento': '2025-05-02 17:51:50', 'servicio': 'Bombero', 'tipo': 'Individual', 'discapacidad': 'Grado 4: Discapacidad grave', 'nivel_emergencia': 'Nivel 2: Emergencia moderada', 'lat': 39.482211, 'lon': -0.418151, 'edad': 17}, {'evento_id': '01991477', 'timestamp_evento': '2025-05-02 17:51:29', 'servicio': 'Bombero', 'tipo': 'Colectiva', 'discapacidad': 'Grado 5: Discapacidad muy grave', 'nivel_emergencia': 'Nivel 1: Emergencia leve', 'lat': 39.439647, 'lon': -0.41478}, {'evento_id': 'bce6f58f', 'timestamp_evento': '2025-05-02 17:51:24', 'servicio': 'Bombero', 'tipo': 'Individual', 'discapacidad': 'Grado 4: Discapacidad grave', 'nivel_emergencia': 'Nivel 3: Emergencia grave', 'lat': 39.484957, 'lon': -0.340855, 'edad': 17}], [{'recurso_id': 31, 'servicio': 'Bombero', 'latitud': 39.455763, 'longitud': -0.37434, 'timestamp_ubicacion': '2025-05-02 17:52:06'}, {'recurso_id': 25, 'servicio': 'Bombero', 'latitud': 36.455763, 'longitud': -0.57434, 'timestamp_ubicacion': '2025-05-02 17:50:06'}]))

In [13]:
import apache_beam as beam
import math
import logging
from datetime import datetime, timedelta, timezone

logging.basicConfig(level=logging.INFO, format="%(message)s")

logging.getLogger("apache_beam").setLevel(logging.WARNING)

In [12]:
def calculo_coeficiente(vehiculo, emergencia):
    servicio = emergencia["servicio"]
    tipo_emergencia = emergencia["tipo"]
    edad=emergencia.get("edad", 0)
    discapacidad=emergencia["discapacidad"]
    nivel_emergencia=emergencia["nivel_emergencia"]
    latitud_emergencia=emergencia["lat"]
    longitud_emergencia=emergencia["lon"]
    latitud_vehiculo=vehiculo["latitud"]
    longitud_vehiculo=vehiculo["longitud"]

    if tipo_emergencia == "Individual":
        tipo_score = 0
        if edad < 10:
            tipo_score = 0.5
        elif edad > 70:
            tipo_score = 0.5
    elif tipo_emergencia == "Colectiva":
        tipo_score = 1.0
    
    if discapacidad == "Grado 1: Discapacidad nula":
        disc_score= 0.0
    elif discapacidad == "Grado 2: Discapacidad leve":
        disc_score= 0.2
    elif discapacidad == "Grado 3: Discapacidad moderada":
        disc_score= 0.4
    elif discapacidad == "Grado 4: Discapacidad grave":
        disc_score= 0.6
    elif discapacidad == "Grado 5: Discapacidad muy grave":
        disc_score= 0.8

    if nivel_emergencia == "Nivel 1: Emergencia leve":
        nivel_score= 0.2
        tiempo_total= 30
    elif nivel_emergencia == "Nivel 2: Emergencia moderada":
        nivel_score= 0.5
        tiempo_total= 60
    elif nivel_emergencia == "Nivel 3: Emergencia grave":
        nivel_score= 0.8
        tiempo_total= 120

    delta_lat = latitud_emergencia - latitud_vehiculo
    delta_lon = longitud_emergencia - longitud_vehiculo
    lat_prom = (latitud_vehiculo + latitud_emergencia) / 2

    # Aprox. metros por grado
    metros_lat = delta_lat * 111_000
    metros_lon = delta_lon * 111_000 * math.cos(math.radians(lat_prom))

    distancia_metros = (metros_lat**2 + metros_lon**2) ** 0.5        
    distancia_score = 1 / (1 + distancia_metros)
    
    if servicio == "Policia":
        tiempo_respuesta =  distancia_metros / 15*60  # Velocidad promedio de un vehículo de policia en m/s, pasado a minutos
    else:
        tiempo_respuesta =  distancia_metros / 11*60  # Velocidad promedio de un vehículo de bomberos o ambulancia en m/s, pasado a minutos

    tiempo_total+=tiempo_respuesta
    

    pesos = {
        "tipo": 0.2,
        "discapacidad": 0.25,
        "nivel": 0.4,
        "distancia": 0.25
    }

    # --- Cálculo del coeficiente ponderado ---
    coeficiente = (
        tipo_score * pesos["tipo"] +
        disc_score * pesos["discapacidad"] +
        nivel_score * pesos["nivel"] +
        distancia_score * pesos["distancia"]
    )

    return round(coeficiente, 4), tiempo_respuesta, tiempo_total, distancia_metros




clave, (emergencias, vehiculos) = Grouped_data
for i, emergencia in enumerate(emergencias):
    coeficinetes_lista = []
    for vehiculo in vehiculos:
        coficiente = calculo_coeficiente(vehiculo, emergencia)[0]
        coeficinetes_lista.append(coficiente)
    emergencia["coeficientes"]=coeficinetes_lista

if clave == "Bombero":
    logging.info(f"Bomberos: ({vehiculos}, {emergencias})")


Bomberos: ([{'recurso_id': 31, 'servicio': 'Bombero', 'latitud': 39.455763, 'longitud': -0.37434, 'timestamp_ubicacion': '2025-05-02 17:52:06'}, {'recurso_id': 25, 'servicio': 'Bombero', 'latitud': 36.455763, 'longitud': -0.57434, 'timestamp_ubicacion': '2025-05-02 17:50:06'}], [{'evento_id': '51c57837', 'timestamp_evento': '2025-05-02 17:51:04', 'servicio': 'Bombero', 'tipo': 'Individual', 'discapacidad': 'Grado 2: Discapacidad leve', 'nivel_emergencia': 'Nivel 2: Emergencia moderada', 'lat': 39.465235, 'lon': -0.380162, 'edad': 56, 'coeficientes': [0.2502, 0.25]}, {'evento_id': 'af8c53ce', 'timestamp_evento': '2025-05-02 17:51:40', 'servicio': 'Bombero', 'tipo': 'Individual', 'discapacidad': 'Grado 1: Discapacidad nula', 'nivel_emergencia': 'Nivel 3: Emergencia grave', 'lat': 39.475542, 'lon': -0.394527, 'edad': 77, 'coeficientes': [0.4201, 0.42]}, {'evento_id': '90a17027', 'timestamp_evento': '2025-05-02 17:51:50', 'servicio': 'Bombero', 'tipo': 'Individual', 'discapacidad': 'Grado 

In [15]:
mensaje=([{'recurso_id': 31, 'servicio': 'Bombero', 'latitud': 39.455763, 'longitud': -0.37434, 'timestamp_ubicacion': '2025-05-02 17:52:06'}, {'recurso_id': 25, 'servicio': 'Bombero', 'latitud': 36.455763, 'longitud': -0.57434, 'timestamp_ubicacion': '2025-05-02 17:50:06'}], [{'evento_id': '51c57837', 'timestamp_evento': '2025-05-02 17:51:04', 'servicio': 'Bombero', 'tipo': 'Individual', 'discapacidad': 'Grado 2: Discapacidad leve', 'nivel_emergencia': 'Nivel 2: Emergencia moderada', 'lat': 39.465235, 'lon': -0.380162, 'edad': 56, 'coeficientes': [0.2502, 0.25]}, {'evento_id': 'af8c53ce', 'timestamp_evento': '2025-05-02 17:51:40', 'servicio': 'Bombero', 'tipo': 'Individual', 'discapacidad': 'Grado 1: Discapacidad nula', 'nivel_emergencia': 'Nivel 3: Emergencia grave', 'lat': 39.475542, 'lon': -0.394527, 'edad': 77, 'coeficientes': [0.4201, 0.42]}, {'evento_id': '90a17027', 'timestamp_evento': '2025-05-02 17:51:50', 'servicio': 'Bombero', 'tipo': 'Individual', 'discapacidad': 'Grado 4: Discapacidad grave', 'nivel_emergencia': 'Nivel 2: Emergencia moderada', 'lat': 39.482211, 'lon': -0.418151, 'edad': 17, 'coeficientes': [0.3501, 0.35]}, {'evento_id': '01991477', 'timestamp_evento': '2025-05-02 17:51:29', 'servicio': 'Bombero', 'tipo': 'Colectiva', 'discapacidad': 'Grado 5: Discapacidad muy grave', 'nivel_emergencia': 'Nivel 1: Emergencia leve', 'lat': 39.439647, 'lon': -0.41478, 'coeficientes': [0.4801, 0.48]}, {'evento_id': 'bce6f58f', 'timestamp_evento': '2025-05-02 17:51:24', 'servicio': 'Bombero', 'tipo': 'Individual', 'discapacidad': 'Grado 4: Discapacidad grave', 'nivel_emergencia': 'Nivel 3: Emergencia grave', 'lat': 39.484957, 'lon': -0.340855, 'edad': 17, 'coeficientes': [0.4701, 0.47]}])

In [ ]:
vehiculos, emergencias = mensaje
match_list_emergencias_id = []
match_list_vehiculos_id = []

while True:
    max_total = -1
    mejor_emergencia = None
    mejor_vehiculo = None
    i_emergencia = -1
    i_vehiculo = -1

    for idx_e, emergencia in enumerate(emergencias):
        if emergencia["evento_id"] in match_list_emergencias_id:
            continue

        for idx_v, vehiculo in enumerate(vehiculos):
            if vehiculo["recurso_id"] in match_list_vehiculos_id:
                continue

            coef = emergencia["coeficientes"][idx_v]
            if coef > max_total:
                max_total = coef
                mejor_emergencia = emergencia
                mejor_vehiculo = vehiculo
                i_emergencia = idx_e
                i_vehiculo = idx_v

    if mejor_emergencia is None or mejor_vehiculo is None:
        break  # Ya no hay emparejamientos posibles

    # Añadir información al match
    mejor_emergencia["coeficiente_seleccionado"] = max_total
    tiempo_dist = calculo_coeficiente(mejor_vehiculo, mejor_emergencia)
    mejor_emergencia["tiempo_total"] = tiempo_dist[2]
    mejor_emergencia["tiempo_respuesta"] = tiempo_dist[1]
    mejor_emergencia["distancia_recorrida"] = tiempo_dist[3]
    mejor_emergencia["disponible_en"] = datetime.now() + timedelta(minutes=mejor_emergencia["tiempo_total"])

    match_list_emergencias_id.append(mejor_emergencia["evento_id"])
    match_list_vehiculos_id.append(mejor_vehiculo["recurso_id"])

    yield beam.pvalue.TaggedOutput("Match", (mejor_vehiculo, mejor_emergencia))

# Emitir emergencias sin emparejar
for emergencia in emergencias:
    if emergencia["evento_id"] not in match_list_emergencias_id:
        yield beam.pvalue.TaggedOutput("NoMatch", emergencia)


Emparejamiento: {'recurso_id': 31, 'servicio': 'Bombero', 'latitud': 39.455763, 'longitud': -0.37434, 'timestamp_ubicacion': '2025-05-02 17:52:06'} - {'evento_id': '01991477', 'timestamp_evento': '2025-05-02 17:51:29', 'servicio': 'Bombero', 'tipo': 'Colectiva', 'discapacidad': 'Grado 5: Discapacidad muy grave', 'nivel_emergencia': 'Nivel 1: Emergencia leve', 'lat': 39.439647, 'lon': -0.41478, 'coeficientes': [0.4801, 0.48], 'coeficiente_seleccionado': 0.4801, 'tiempo_total': 21306.461252576115, 'tiempo_respuesta': 21276.461252576115, 'distancia_recorrida': 3900.684562972288, 'disponible_en': datetime.datetime(2025, 5, 17, 15, 25, 59, 328805)}
Emparejamiento: {'recurso_id': 25, 'servicio': 'Bombero', 'latitud': 36.455763, 'longitud': -0.57434, 'timestamp_ubicacion': '2025-05-02 17:50:06'} - {'evento_id': 'bce6f58f', 'timestamp_evento': '2025-05-02 17:51:24', 'servicio': 'Bombero', 'tipo': 'Individual', 'discapacidad': 'Grado 4: Discapacidad grave', 'nivel_emergencia': 'Nivel 3: Emergen